In [25]:
import numpy as np
from bitstring import BitArray
import random
np.set_printoptions(suppress = True, threshold = np.inf, precision = 15)
import bitstring, random 

In [26]:
#shift orchest模块
def shiftOrchest_moudle(w_manti_list):
    sewo = [[""] * 64 for _ in range(8)]    #8*64
    for i in range(7,-1,-1):
        for j in range(64):
            sewo[7-i][j] = w_manti_list[j][i]
    return sewo


In [27]:
#无符号数移位 将8位无符号数，右移shift_num位，扩充成31位
#              8位宽active   位移   最终扩展成31位宽  
def uint_shift(source_num, shift_num):
    shift_l =f"{'0' * (23 - shift_num)}{source_num}{'0' * shift_num}"
    result = int(shift_l, 2)  # 指定基数为 2，表示这是一个二进制字符串
    return result

In [28]:
                    #decoder模块
def decoder_moudle(sewo,a_manti,sign,i):

    result_num = 0
    result_num_list = []

    for b in range(8):      #第几个窗口
        result_num = 0
        for c in range(8):  #窗口里的哪个数
            if int(sewo[8 * b + c]) == 1 :
                #如果符号是正，则经行加法，如果符号是负，则进行减法
                result_num = (result_num +(uint_shift(a_manti[8 * b + c],i))) if (sign[8 * b + c] == 0) else (result_num -(uint_shift(a_manti[8 * b + c],i)))
        
        result_num_list.append(result_num)

    return result_num_list

In [29]:
                        #adder tree模块
def adder_Tree_moudle(outnum):
    result1 = 0
    result2 = 0
    for a in range(8):  #8个swewo
        print("outnum=",outnum[a])
        #本意模仿chisel加法树，实际在python没法实现
        for b in range(0,4):
            result1 = result1 + outnum[a][b]
        for b in range(4,8):
            result2 = result2 + outnum[a][b]
    result= result1+result2
    return result

In [30]:
                     #输入预处理
import numpy as np

              # T2

weight = np.array([
39,61,-85,109,-30,-88,76,-7,-97,
15,-106,-26,-15,118,-43,7,69,
-48,-26,30,-34,76,-48,16,126,
-3,73,-75,-50,-42,-56,55,57,
3,-91,74,-106,-128,-15,34,-23,
-38,31,-15,-9,60,125,-85,-117,
71,-13,10,-6,46,-26,-98,-27,
33,67,-112,-70,-2,89,-62

])


active = np.array([
63,100,-51,87,-72,-22,-68,-85,124,
47,56,81,-95,22,35,4,-64,
-27,32,-110,33,-37,89,98,86,
-51,66,-82,121,15,21,73,0,
-98,46,5,30,60,-50,-54,115,
42,-46,-38,-33,-40,-91,-40,44,
82,-102,54,-61,95,-43,-54,-109,
11,105,65,-1,5,11,68

       ])


'''

weight = [random.randint(-128, 127) for _ in range(64)]
active = [random.randint(-128, 127) for _ in range(64)]
'''
                     #软件模拟结果   
mac_result_software = 0
for a in range(64):
    multiply_mid = weight[a] * active[a]
    mac_result_software = multiply_mid + mac_result_software
print('the mac result is ',mac_result_software)

                     #预处理           

exp = [0] *64
weight_int8 = [""] * 64
active_int8 = [""] * 64
a_sign = [""] * 64
w_sign = [""] * 64
sign   = [""] * 64

sign = [""] * 64
for b in range(64): 
       weight_int8[b] = '{0:08b}'.format(abs(weight[b]))
       active_int8[b] ='{0:08b}'.format(abs(active[b]))   
       if weight[b] >= 0:
              w_sign[b] = 0
       else:
              w_sign[b] = 1

       if active[b] >= 0:
              a_sign[b] = 0
       else:
              a_sign[b] = 1
       if w_sign[b]^a_sign[b]:
              sign[b] = 1
       else:
              sign[b] = 0
w_manti_list = []
a_manti_list = []

for c in range(64):
       w_manti_list.append(weight_int8[c])
       a_manti_list.append(active_int8[c])

sewo = shiftOrchest_moudle(w_manti_list)

outnum = []
for d in range(8):
       outnum.append(decoder_moudle(sewo[d],a_manti_list,sign,d))

result = adder_Tree_moudle(outnum)

print('the bitlet result is ',result)


the mac result is  77884
outnum= [-102, -76, -203, 13, 13, 7, -150, -206]
outnum= [-80, 168, 194, 32, 328, -34, -160, -52]
outnum= [-72, 64, 96, 392, 1652, -156, 312, -352]
outnum= [1832, 72, 728, -40, 880, -288, -656, 144]
outnum= [2016, 432, 656, -976, 7216, -2272, -704, -1296]
outnum= [9248, 64, 1408, -3872, 7680, -6080, 3712, -5280]
outnum= [8576, 8640, 2048, 9216, 19648, 1152, -4352, 16704]
outnum= [0, 0, 0, 0, 0, 0, 0, 0]
the bitlet result is  77884
